# Demo 5: Collaborative Filtering and Comedy! 
------
<img src="images/seinfeld.jpg" width="400" height="400">

#### Real Dataset: http://eigentaste.berkeley.edu/dataset/ Dataset 2 
#### Rate Jokes: http://eigentaste.berkeley.edu

## What are we trying to learn from this dataset?

# QUESTION:  Can Collaborative Filtering be used to find which jokes to recommend to our users?


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
import pandas
import cassandra
import pyspark
import re
import os
import matplotlib.pyplot as plt
from IPython.display import IFrame
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
%store -r astraUsername astraPassword astraSecureConnect astraKeyspace

#### Helper function to have nicer formatting of Spark DataFrames

In [7]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  10, truncate = False):
    if(truncate):
        pandas.set_option('display.max_colwidth', 100)
    else:
        pandas.set_option('display.max_colwidth', None)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

## Creating Tables and Loading Tables

### Connect to Cassandra

In [8]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    'secure_connect_bundle': '/tmp/'+astraSecureConnect
}
auth_provider = PlainTextAuthProvider(username=astraUsername, password=astraPassword)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

### Set keyspace 

In [9]:
session.set_keyspace(astraKeyspace)

### Create table called jokes. Our PRIMARY will need to be a unique composite key (userid, jokeid). This will result in an even distribution of the data and allow for each row to be unique. Remember we will have to utilize that PRIMARY KEY in our WHERE clause in any of our CQL queries. 

In [10]:
query = "CREATE TABLE IF NOT EXISTS jokes \
                                    (userid int, jokeid int, rating float, \
                                     PRIMARY KEY (userid, jokeid))"
session.execute(query)

### What do these of these 3 columns represent: 

* **Column 1**: User id
* **Column 2**: Joke id
* **Column 3**: Rating of joke (-10.00 - 10.00) 

In [11]:
#download file to local (working on better way)
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket('andygoade-dev')

blob = storage.Blob('notebooks/jupyter/data/jester_ratings3.csv', bucket)
blob.download_to_filename('/tmp/jester_ratings3.csv')

### Load Jokes dataset from CSV file (jester_ratings3.csv)
* This is a file I created from the *.dat file and I only have 10,000 rows -- dataset has over 1 million rows
<img src="images/laughing.gif" width="300" height="300">

#### Insert all the Joke Rating Data into the table `jokes`

In [12]:
fileName = '/tmp/jester_ratings3.csv'
input_file = open(fileName, 'r')

for line in input_file:
    jokeRow = line.split(',')
    query = "INSERT INTO jokes (userid, jokeid, rating)"
    
    query = query + "VALUES (%s, %s, %s)"
    
    session.execute(query, (int(jokeRow[0]), int(jokeRow[1]) , float(jokeRow[2]) ))

#### Do a select * on joke_table WHERE userid = x to verify that data was loaded into the table

In [13]:
query = 'SELECT * FROM jokes WHERE userid = 65'
rows = session.execute(query)
for row in rows:
    print (row.userid, row.jokeid, row.rating)

65 5 9.875
65 7 9.718999862670898
65 8 10.0
65 13 8.625
65 15 7.406000137329102
65 16 -9.875
65 17 -9.531000137329102
65 18 -8.5
65 19 -7.156000137329102
65 20 -0.09399999678134918
65 21 6.938000202178955
65 22 10.0
65 23 9.937999725341797
65 24 6.843999862670898
65 25 10.0
65 26 9.968999862670898
65 27 3.2809998989105225
65 28 9.968999862670898
65 29 2.0309998989105225
65 30 10.0
65 31 6.938000202178955
65 32 2.937999963760376
65 33 10.0
65 34 10.0
65 35 5.75
65 36 2.875
65 37 10.0
65 38 10.0
65 39 9.937999725341797
65 40 9.968999862670898
65 41 10.0
65 42 9.968999862670898
65 43 10.0
65 44 9.875
65 45 9.968999862670898
65 46 9.812000274658203
65 47 10.0
65 48 6.938000202178955
65 49 6.938000202178955
65 50 1.0
65 51 10.0
65 52 9.968999862670898
65 53 6.0
65 54 6.938000202178955
65 55 9.937999725341797
65 56 10.0
65 57 9.875
65 58 9.875
65 59 10.0
65 60 10.0
65 61 6.938000202178955
65 62 6.938000202178955
65 63 9.937999725341797
65 64 9.937999725341797
65 65 6.938000202178955
65 66 6.

<img src="images/sparklogo.png" width="150" height="200">

### Finally time for Apache Spark! 

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [14]:
spark = SparkSession \
    .builder \
    .appName('demo5') \
    .master("local") \
    .config( \
        "spark.cassandra.connection.config.cloud.path", \
        "file:/tmp/"+astraSecureConnect) \
    .config("spark.cassandra.auth.username", astraUsername) \
    .config("spark.cassandra.auth.password", astraPassword) \
    .getOrCreate()

jokeTable = spark.read.format("org.apache.spark.sql.cassandra").options(table="jokes", keyspace=astraKeyspace).load()

print ("Table Row Count: ")
print (jokeTable.count())

Table Row Count: 
10000


#### Split dataset into training and testing set 

In [15]:
(training, test) = jokeTable.randomSplit([0.8, 0.2])

training_df = training.withColumn("rating", training.rating.cast('int'))
testing_df = test.withColumn("rating", test.rating.cast('int'))

showDF(training_df)

,userid,jokeid,rating
0,4,5,-5
1,4,8,-4
2,9,5,8
3,9,7,9
4,9,8,8
5,9,13,9
6,9,15,9
7,9,16,9
8,9,17,9
9,9,18,8


### Setup for CFliter with ALS

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [16]:
als = ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="jokeid", ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training_df)

In [17]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(testing_df)

# Generate top 10 joke recommendations for each user
userRecs = model.recommendForAllUsers(10)

showDF(userRecs)

# Generate top 10 user recommendations for each joke
jokeRecs = model.recommendForAllItems(10)

,userid,recommendations
0,148,"[(94, 48.496822357177734), (67, 28.267776489257812), (120, 22.04917335510254), (52, 21.876514434814453), (125, 19.757862091064453), (60, 19.37693977355957), (28, 18.72410774230957), (146, 12.179214477539062), (96, 11.932487487792969), (86, 10.769506454467773)]"
1,243,"[(97, 47.78932189941406), (63, 39.99330520629883), (59, 36.94940948486328), (41, 36.36469650268555), (28, 35.27206039428711), (131, 29.883909225463867), (84, 29.622997283935547), (48, 28.542932510375977), (68, 28.090585708618164), (60, 28.072216033935547)]"
2,251,"[(50, 4.929049968719482), (54, 2.617748260498047), (127, 1.8731317520141602), (139, 1.147890567779541), (123, 1.1444554328918457), (121, 0.44882816076278687), (36, -0.17334342002868652), (28, -0.2526559829711914), (122, -0.5632102489471436), (35, -0.7104265689849854)]"
3,85,"[(94, 126.17559051513672), (67, 70.26445007324219), (26, 68.96479034423828), (116, 61.49433135986328), (127, 58.935401916503906), (96, 58.466426849365234), (123, 56.596092224121094), (119, 54.1373176574707), (60, 53.99386978149414), (83, 53.864295959472656)]"
4,137,"[(44, 10.958529472351074), (26, 9.489072799682617), (39, 9.216965675354004), (7, 8.996129989624023), (138, 8.972553253173828), (51, 8.782771110534668), (82, 8.562628746032715), (52, 8.411686897277832), (58, 8.410223007202148), (57, 8.281982421875)]"
5,65,"[(59, 11.904026985168457), (22, 11.426578521728516), (116, 11.42410659790039), (84, 10.992903709411621), (45, 10.948541641235352), (105, 10.893614768981934), (85, 10.858660697937012), (34, 10.784358978271484), (26, 10.772669792175293), (77, 10.754459381103516)]"
6,53,"[(108, 9.442584037780762), (119, 8.682197570800781), (61, 8.283303260803223), (71, 8.248950004577637), (46, 7.393714427947998), (109, 7.022269248962402), (64, 7.02130126953125), (34, 6.896993637084961), (105, 6.719516754150391), (54, 6.5335283279418945)]"
7,133,"[(108, 14.529730796813965), (56, 10.37831974029541), (76, 8.843097686767578), (53, 8.431449890136719), (63, 8.231698036193848), (68, 8.143494606018066), (129, 7.98358154296875), (97, 7.729097366333008), (48, 7.647373199462891), (46, 7.62947940826416)]"
8,155,"[(108, 36.55143737792969), (127, 34.43547058105469), (119, 30.052030563354492), (46, 28.394855499267578), (109, 27.94203758239746), (34, 26.703304290771484), (26, 26.34037208557129), (114, 25.51360321044922), (33, 25.476598739624023), (53, 24.883338928222656)]"
9,108,"[(94, 79.65190887451172), (55, 56.78767013549805), (52, 50.802162170410156), (116, 45.264564514160156), (28, 41.672584533691406), (127, 34.99951171875), (67, 34.596412658691406), (60, 32.1795654296875), (138, 30.048805236816406), (103, 25.85186767578125)]"


In [18]:
showDF(userRecs.filter(userRecs.userid == 65))

,userid,recommendations
0,65,"[(59, 11.904026985168457), (22, 11.426578521728516), (116, 11.42410659790039), (84, 10.992903709411621), (45, 10.948541641235352), (105, 10.893614768981934), (85, 10.858660697937012), (34, 10.784358978271484), (26, 10.772669792175293), (77, 10.754459381103516)]"


In [19]:
IFrame(src='images/init94.html', width=700, height=200)

In [20]:
IFrame(src='images/init43.html', width=700, height=200)

In [21]:
session.execute("""drop table jokes""")

In [22]:
spark.stop()